<img src="TwisterII.png">
<img src="ABayesian.png">
<center><i>Lehman Garrison, George Miller, Stephen Portillo</i></center>

Most data files and intermediate files can be found at our Github: https://github.com/lgarrison/AM207-Twister

Some raw data files were too big to include in the repository and are available upon request.

# Abstract
Tornadoes cost dozens of lives and billions of dollars in the United States every year.  Significant time, money, and effort is spent preparing for these natural disasters, so understanding their geographical distribution is key for allocation of resources.  To address the question of tornado frequency and intensity, we construct a Bayesian hierarchical model for the occurrence of tornadoes by county in the United States.  The grouping of counties into states, and tornadoes into annual seasons, provide a natural hierarchical structure that we exploit in our model.  We use historical tornado track data from 1950-2014 and Hamiltonian Monte Carlo to sample from the posterior distribution of model parameters.  We construct a second hierarchical model for the distribution of injuries due to tornadoes on a state and year level.  Our hierarchical approach in both cases allows us to distinguish factors like year-to-year variations from state-to-state variations, which could be an important indicator of whether a state government has effective tornado preparedness systems in place.  Finally, we incorporate a dataset of tornado warnings into our injury model to explore whether tornado warnings are effective at saving lives, and we find no statistically significant evidence that they do.  However, we may be limited by data incompleteness and imperfectness in matching warnings to tornado events.

# Introduction
Tornadoes are the have the fastest winds of any atmospheric storms, the strongest storms reaching wind speeds of more than 300 mph. These violent winds destroy structures and turn the resulting debris into hazardous projectiles. These storms' great destructive power and meteorologists' efforts to understand them have captured the public imagination, like in Jan de Bont's 1996 film _Twister_. Tornadoes occur all over the world but are most common in Tornado Alley, making tornado preparedness a great public concern in the United States. Much effort has been put into improving tornado forecasts, warning systems, and emergency infrastructure.

We approach the problem of forecasting the frequency, injury rate, and fatality rate of tornadoes in the United States using historical data. Such forecasts are important for all levels of government in terms of financial planning and emergency logistics planning. Our models distinguish geographical and yearly. Geographical effects on tornado frequency capture where conditions are most favourable to tornado formation; geographical effects on tornado injuries and fatalities may reflect the tornado preparedness of different regions, such as the availability of sufficient shelter. Yearly effects on tornado frequency capture possible long-term climatological shifts, while yearly effects on tornado injuries and fatalities may reflect nation-wide improvements in building codes or public awareness. Also, in modelling tornado injuries and fatalities, we search for any effect National Weather Service warnings may have in reducing losses.

# Related Work

Previous analyses have used historical to find climatological variables that can predict tornado frequency across the United States.  Wilke and Anderson (http://onlinelibrary.wiley.com/doi/10.1029/2002JD002806/pdf) use a hierarchical Bayesian model to correct for tornado reporting errors and tornado correlations. They find that an index of El Nino activity predicts tornado frequency in a regionally-dependent way. Cheng et al. (http://www.nature.com/ncomms/2015/150325/ncomms7599/full/ncomms7599.html) instead model tornado frequency using monthly-averaged atmospheric variables measured across the nation. They find that in each season of the year, different variables are best at predicting tornado frequency. Their model captures a shift in tornado activity from the Northern Great Plains to the Canadian Prairies which occurs during the summer. 

These analyses identify climatological variables that can be measured today to forecast the occurence of tornadoes this year. By contrast, we identify the years when tornado frequency is higher or lower, but we do not identify measurements that could be made to forecast whether 2015 will be a good or bad year. Instead, we are concerned with the long-term average tornado frequency, injuries, and fatalities that different locations will experience.

# Methods

## Data
The National Weather Service’s Storm Prediction Center (http://www.spc.noaa.gov/wcm/) maintains a historical tornado record, including F scale intensity, counties along the track, and reported injuries. The Extended Fujita scale (F scale) tornadoes into one of six categories (F0-5) based on the damage they cause. The F scale can be used as a proxy for a tornado's intrinsic strength, ie. its wind speed.

We use county land area and population as reported in the United States Census (http://factfinder.census.gov/).

The Iowa Environmental Mesonet maintains a historical tornado warning record (https://mesonet.agron.iastate.edu/request/gis/watchwarn.phtml). We want to include in our injury model whether a tornado warning was issued for each tornado, and if so, the time between the warning's issuence and the tornado's touchdown. These data do not exist in a single public dataset, so we reconstruct the warning time from the tornado record and tornado warning record.  Our procedure is as follows:

1. Aggregate county-level tornado warnings from 1986 onwards [Count: 224565 warnings]
2. Aggregate tornado tracks from 1986 onwards [Count: 34126 tornadoes]
3. Perform a Pandas merge (equivalent to a SQL JOIN) between the two using the counties as keys [Count: 3493362 matches]
4. Limit the matches to touchdowns that occur up to 2 hours before the issuance of the warning and 1 hour after the expiry of the warning [Count: 51292]
5. Choose the warning with the largest warning time for a tornado [Count: 24444 tornadoes]
6. Save: (a) whether a tornado warning was issued, and (b) the warning time

From 2006 onward, we can also perform the same procedure with tornado watches. See the [Tornado Warning Pipeline Notebook](tornado_warnings_pipeline.ipynb) for more details.


## Modelling Approach
We employ a Bayesian scheme of hierarchical linear modeling, in which parameter dependences can be constructed on multiple structured levels. These multilevel models are advantageous when examining nested data, such as the occurrence of tornados on a county, state, and nation-wide level. From a basic level, the tornado rates for each county will be modeled by a Poisson distribution given some rate parameter.  The advantage of a multi-level model is that each county can have it’s own mid-level rate parameter, while each of these can then be influenced by a state-level (or any larger geographical area) hyperparameter.  Thus, even though no tornados may have ever touched down in a specific Iowan county, the chance of tornadoes will still be very high since the state of Iowa is in Tornado Alley.

## Library Used: Stan
To build our model, we use the probabilistic programming language Stan (http://mc-stan.org/). Stan uses a “no U-turn” (NUTS) Hamiltonian Monte Carlo sampler. Hamiltonian Monte Carlo models the traversal of the posterior as a dynamics problem in order to adaptively refine the step size, and is especially suited to the posterior distributions created by hierarchical models. NUTS automatically tunes the number of leapfrog steps per sample for maximum efficiency.

Sampling from the posterior of this 10,000 dimensional space was a computationally difficult task.  Despite our use of Stan, even 1000 iterations on 7 states could take upwards of 3 hours, and ideally we would like to run many chains, with thousands of iterations each, on all 50 states.  To address this, we tried moving our computations to the Odyssey cluster, since Stan has built-in multiprocessor support.  Despite successfully configuring our code to run on Odyssey and updating all of Odyssey's Python libraries, the performance was disappointing.  On a core-by-core basis, Odyssey was about 8 times slower.  We contacted Odyssey's technical support team, who confirmed that this single-core performance is not surprising, especially since modern desktops are capable of overclocking a single core to accelerate single-threaded tasks, which Odyssey is not designed to do.  We were able to overcome this limitation to some degree by running 32 PyStan processes in parallel on Odyssey, which was an equivalent speedup of 4x, but still a far cry from the 32x we might have hoped for.

# Results
_The Results and Discussion sections in this notebook are summaries of the [Tornado Frequency Modelling Notebook](Tornado_Model.ipynb) and [Tornado Injury Modelling Notebook](injurymodelling.ipynb)._

## Tornado Frequency

To model the frequency of tornados, we use a Poisson generalized linear model. The log Poisson rate of tornadoes in a county is the sum of the county factor, the county’s log land mass, the year factor for the year of occurence, and an additional normal error term to allow for overdispersion.  Further, the county-level factor is normally dependent on additional hyperpriors corresponding to its respective US state.  Finally, all parameters are separated by possible tornado F-scales.  All priors are either uniform (for means), or cauchy (for variances).

## Tornado Injuries and Fatalities

To model the injuries (and fatalities) from tornadoes, we use a negative binomial generalized linear model. The negative binomial is a Poisson with overdispersion set by a precision parameter. The log expected number of injuries from a tornado is the sum of the state factor, the affected area's log population density times a power-law index, a year factor for the year of occurence, and an additional normal error term to allow for overdispersion.  The variance hyperparameters and the negative binomial precision are given Cauchy priors. Finally, all parameters are separated by possible tornado F-scales and for injuries vs fatalities.

# Discussion
_The Results and Discussion sections in this notebook are summaries of the [Tornado Frequency Modelling Notebook](Tornado_Model.ipynb) and [Tornado Injury Modelling Notebook](injurymodelling.ipynb)._

## Tornado Frequency

The year factors suggest that the number of F0-1 tornadoes has increased and the number of F2+ tornadoes has decreased. The increase in the number of F0-1 tornadoes is probably a completeness effect: improvements in tornado detection have allowed us to report more of the weak tornadoes. Part of the change in the number of F2+ tornadoes may be from redefinitions of the Fujita scale - the Modified Fujita scale in 1992 and the Enhanced Fujita scale in 2007. An actual decrease in the F2+ tornado rate seems unlikely but cannot be ruled out.

The county factors reflect the relative frequency of tornadoes in each county. Our hierarchical model assumes that all county factors in a state are drawn from some Gaussian population that depends on the state, effectively smoothing tornado frequency at the state level. Future work could use natural meteorlogically-relevant boundaries or perhaps just a latitude-longitude grid to define regions to group counties into. Separately modelling each F class of tornado allows us to see the different geographical distribution of weak tornadoes vs strong tornadoes.

## Tornado Injuries and Fatalities

Tornado injuries and fatalities are not proportional to the population density, but instead follow a power law of population density. For F3 and F4 tornadoes, injuries go with the $0.5$ power of population density while fatalities go with the $0.25$ power. The negative binomial generalized linear model passes model checks - it generates injury and fatality distributions which look similar to the data.

We do not find strong evidence that tornado warnings prevent injuries or fatalities. We may be limited by imperfections in how we match tornadoes to warnings, and also by the fact we do not have warning data for the entire time range we study.

While most state factors are found to be consistent with zero, we find some significant state factors. Some Southern states appear to have more injuries and fatalities than average from F3 tornadoes - possibly because these same states have a high fraction of mobile homes (http://www.spc.noaa.gov/gis/svrgis/images/mobile_home.png). The error bars on the state factors for F4 and F5 tornadoes are harder to interpret because the error bars are larger.

The year factors have very large error bars and are consistent with zero.

# Conclusion

We used hierarchical generalized linear regression to model the frequency of tornadoes and the number of injuries and fatalities they cause, separating year-to-year differences from geographical differences.

In modelling tornado frequency, the yearly factors seem to reflect changes in tornado detection and categorization. The county factors identify counties where tornadoes can be expected more frequently, and the hierarchical model serves to smooth tornado frequency at the state level.

In modelling tornado injuries (and fatalities), we find that power law dependences on population density reproduce the observed injury distribution well. We do not find evidence that warnings have any effects on injury numbers. We find some evidence of state-level effects on F3 tornado injuries and fatalities.

Our project shows that using hierarchical linear models for tornado properties is computationally feasible. These models may be made more sensitive to yearly and geographical effects by including more data. Instead of pooling counties by state, counties could be pooled using natural features like mountain ranges. And instead of using the average population density of the county in which a tornado touches down, it may be useful to look up the total population that lives within a certain radius of that tornado's track.